In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

In [3]:
final_df = pd.read_csv("../half_season/h1_dfs/final_transition_out_of_poss_df_H1.csv")
final_df = final_df.rename(columns={'Unnamed: 0': 'uniqueTeamId'})
team_ids_h1 = final_df['uniqueTeamId']
final_df.set_index("uniqueTeamId", inplace = True)
final_df = final_df.fillna(0)

final_df_h2 = pd.read_csv("../half_season/h2_dfs/final_transition_out_of_poss_df_H2.csv")
final_df_h2 = final_df.rename(columns={'Unnamed: 0': 'uniqueTeamId'}).reset_index()
team_ids_h2 = final_df_h2['uniqueTeamId']
final_df_h2 = final_df_h2.fillna(0)
final_df_h2.set_index("uniqueTeamId", inplace = True)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from kmedoids import KMedoids
from pyclustering.cluster.kmedoids import kmedoids
from pyclustering.cluster import cluster_visualizer
from pyclustering.utils import calculate_distance_matrix
import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

    # Autoencoder for DEC
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim=10):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )

    def forward(self, x):
        return self.model(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, z):
        return self.model(z)

class AutoEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim=10):
        super().__init__()
        self.encoder = Encoder(input_dim, latent_dim)
        self.decoder = Decoder(latent_dim, input_dim)

    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out


    # DEC model
class DEC(nn.Module):
    def __init__(self, encoder, cluster_centers):
        super().__init__()
        self.encoder = encoder
        self.clusters = nn.Parameter(cluster_centers)

    def forward(self, x):
        z = self.encoder(x)
        q = 1.0 / (1.0 + torch.sum((z.unsqueeze(1) - self.clusters)**2, dim=2))
        q = q / torch.sum(q, dim=1, keepdim=True)
        return q

# Target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.t() / weight.sum(1)).t()

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist

def compute_indices(X, labels):
    n_clusters = len(np.unique(labels))
    cluster_centers = []
    for i in range(n_clusters):
        cluster_points = X[labels == i]
        if len(cluster_points) == 0:
            cluster_centers.append(np.zeros(X.shape[1]))
        else:
            cluster_centers.append(cluster_points.mean(axis=0))
    cluster_centers = np.array(cluster_centers)

    distances = pairwise_distances(X, cluster_centers)

    # Iwcss: Sum of squared distances to cluster centers
    Iwcss = sum(np.sum((X[labels == i] - cluster_centers[i]) ** 2) for i in range(n_clusters))

    # Isep: Mean distance between all cluster centers
    Isep = np.mean(cdist(cluster_centers, cluster_centers))

    # Idistcc and Idens with filtering for valid clusters
    valid_dists = []
    valid_dens = []
    for i in range(n_clusters):
        cluster_points = X[labels == i]
        if len(cluster_points) < 2:
            continue  # Skip small clusters to avoid NaN in mean/std
        distances_to_center = np.linalg.norm(cluster_points - cluster_centers[i], axis=1)
        valid_dists.append(np.mean(distances_to_center))
        valid_dens.append(np.std(distances_to_center))

    # Default to 0 if all clusters were invalid
    Idistcc = np.mean(valid_dists) if valid_dists else 0.0
    Idens = np.mean(valid_dens) if valid_dens else 0.0

    return Iwcss, Isep, Idistcc, Idens



def normalize(val, min_val, max_val, larger_is_better):
    range_ = max_val - min_val
    if np.isclose(range_, 0):
        return 1.0  # or 0.5, depending on how you want to treat flat metrics
    if larger_is_better:
        return np.clip((val - min_val) / (range_ + 1e-10), 0, 1)
    else:
        return np.clip(1 - (val - min_val) / (range_ + 1e-10), 0, 1)

In [5]:
import random
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# If using GPU
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

scaler = StandardScaler()
X_h1 = scaler.fit_transform(final_df)          # FIT on H1
X_h2 = scaler.transform(final_df_h2)           # TRANSFORM H2

pca2 = PCA(n_components=2, random_state=42)
X_h1_pca = pca2.fit_transform(X_h1)            # FIT on H1
X_h2_pca = pca2.transform(X_h2)                # TRANSFORM H2

X_h1_tensor = torch.tensor(X_h1, dtype=torch.float32)
X_h2_tensor = torch.tensor(X_h2, dtype=torch.float32)



dec_loss = {}

from sklearn.metrics import pairwise_distances_argmin

def predict_via_centroids(X_train, y_train, X_eval):
    centers = np.vstack([X_train[y_train == c].mean(axis=0) for c in np.unique(y_train)])
    return pairwise_distances_argmin(X_eval, centers)  # nearest center id

In [6]:
# Store results
results = []

for k in range(2, 11):
    clusterings = {
        "kmeans": KMeans(n_clusters=k, random_state=42),
        "kmedoids": KMedoids(n_clusters=k, metric="euclidean", init="random", max_iter=300, random_state=42),
        "ward": AgglomerativeClustering(n_clusters=k, linkage="ward"),
    }

    # ====== RAW space (scaled) ======
    for method_name, model in clusterings.items():
        # fit on H1
        y_h1 = model.fit_predict(X_h1)

        # get H2 labels
        if hasattr(model, "predict"):
            y_h2 = model.predict(X_h2)
        else:
            y_h2 = predict_via_centroids(X_h1, y_h1, X_h2)  # Ward

        # evaluate on H2
        sil = silhouette_score(X_h2, y_h2)
        Iwcss, Isep, Idistcc, Idens = compute_indices(X_h2, y_h2)

        results.append({
            "k": k,
            "method": method_name,
            "silhouette_score": sil,
            "Iwcss": Iwcss,
            "Isep": Isep,
            "Idistcc": Idistcc,
            "Idens": Idens
        })

    # ====== PCA space ======
    for method_name, model in clusterings.items():
        y_h1 = model.fit_predict(X_h1_pca)

        if hasattr(model, "predict"):
            y_h2 = model.predict(X_h2_pca)
        else:
            y_h2 = predict_via_centroids(X_h1_pca, y_h1, X_h2_pca)

        sil = silhouette_score(X_h2_pca, y_h2)
        Iwcss, Isep, Idistcc, Idens = compute_indices(X_h2_pca, y_h2)

        results.append({
            "k": k,
            "method": f"{method_name}_pca",
            "silhouette_score": sil,
            "Iwcss": Iwcss,
            "Isep": Isep,
            "Idistcc": Idistcc,
            "Idens": Idens
        })


In [7]:
for k in range(2, 11):
    print("---"*10)
    print(f"k == {k}")
    input_dim = X_h1.shape[1]
    latent_dim = 10

    ae = AutoEncoder(input_dim, latent_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(ae.parameters(), lr=1e-3)

    print("AE training...")
    # AE pretrain on H1
    for epoch in range(3000):
        optimizer.zero_grad()
        out = ae(X_h1_tensor)
        loss = criterion(out, X_h1_tensor)
        loss.backward()
        optimizer.step()        
        if epoch % 500 == 0:
            print(f"Epoch: {epoch}")

    # init clusters using H1 latent
    z_h1 = ae.encoder(X_h1_tensor).detach().numpy()
    km = KMeans(n_clusters=k, random_state=42).fit(z_h1)
    init_centers = torch.tensor(km.cluster_centers_, dtype=torch.float32)

    dec = DEC(ae.encoder, init_centers.clone())
    dec_opt = optim.Adam(dec.parameters(), lr=1e-3)

    print("DEC training...")
    # DEC train on H1
    for epoch in range(3000): 
        q = dec(X_h1_tensor)
        p = target_distribution(q.detach())
        kl = torch.nn.functional.kl_div(q.log(), p, reduction="batchmean")
        dec_opt.zero_grad()
        kl.backward()
        dec_opt.step()
        if epoch % 500 == 0:
            print(f"Epoch: {epoch}")

    # ===== evaluate on H2 =====
    with torch.no_grad():
        z_h2 = dec.encoder(X_h2_tensor).cpu().numpy()
        q_h2 = dec(X_h2_tensor)
        y_h2 = torch.argmax(q_h2, dim=1).cpu().numpy()

    sil = silhouette_score(z_h2, y_h2)
    Iwcss, Isep, Idistcc, Idens = compute_indices(z_h2, y_h2)

    results.append({
        "k": k,
        "method": "dec",
        "silhouette_score": sil,
        "Iwcss": Iwcss,
        "Isep": Isep,
        "Idistcc": Idistcc,
        "Idens": Idens
    })


------------------------------
k == 2
AE training...
Epoch: 0
Epoch: 500
Epoch: 1000
Epoch: 1500
Epoch: 2000
Epoch: 2500
DEC training...
Epoch: 0
Epoch: 500
Epoch: 1000
Epoch: 1500
Epoch: 2000
Epoch: 2500
------------------------------
k == 3
AE training...
Epoch: 0
Epoch: 500
Epoch: 1000
Epoch: 1500
Epoch: 2000
Epoch: 2500
DEC training...
Epoch: 0
Epoch: 500
Epoch: 1000
Epoch: 1500
Epoch: 2000
Epoch: 2500
------------------------------
k == 4
AE training...
Epoch: 0
Epoch: 500
Epoch: 1000
Epoch: 1500
Epoch: 2000
Epoch: 2500
DEC training...
Epoch: 0
Epoch: 500
Epoch: 1000
Epoch: 1500
Epoch: 2000
Epoch: 2500
------------------------------
k == 5
AE training...
Epoch: 0
Epoch: 500
Epoch: 1000
Epoch: 1500


KeyboardInterrupt: 

In [ ]:
# Min-max normalization function
def minmax_normalize(val, min_val, max_val, larger_is_better):
    if np.isclose(max_val - min_val, 0):
        return 1.0
    score = (val - min_val) / (max_val - min_val)
    return score if larger_is_better else 1 - score

results_normalized = []

# Compute global min and max values for each metric
index_min = {
    'Iwcss': min(row['Iwcss'] for row in results),
    'Isep': min(row['Isep'] for row in results),
    'Idistcc': min(row['Idistcc'] for row in results),
    'Idens': min(row['Idens'] for row in results),
}

index_max = {
    'Iwcss': max(row['Iwcss'] for row in results),
    'Isep': max(row['Isep'] for row in results),
    'Idistcc': max(row['Idistcc'] for row in results),
    'Idens': max(row['Idens'] for row in results),
}

for row in results:
    # Min-max normalization
    Iwcss_m = minmax_normalize(row['Iwcss'], index_min['Iwcss'], index_max['Iwcss'], larger_is_better=False)
    Isep_m = minmax_normalize(row['Isep'], index_min['Isep'], index_max['Isep'], larger_is_better=True)
    Idistcc_m = minmax_normalize(row['Idistcc'], index_min['Idistcc'], index_max['Idistcc'], larger_is_better=False)
    Idens_m = minmax_normalize(row['Idens'], index_min['Idens'], index_max['Idens'], larger_is_better=False)

    AC1_m = (Iwcss_m + Isep_m + Idistcc_m + Idens_m) / 4
    AC2_m = (1 * Iwcss_m + 0.5 * Isep_m + 1 * Idistcc_m + 0.25 * Idens_m) / 2.75

    results_normalized.append({
        'k': row['k'],
        'method': row['method'],
        'silhouette_score': row['silhouette_score'],
        #'AC1_zscore': AC1_z,
        #'AC2_zscore': AC2_z,
        'AC1': AC1_m,
        'AC2': AC2_m
    })

pd.set_option('display.max_rows', 500)
results_zscore_df = pd.DataFrame(results_normalized)
results_zscore_df.sort_values(by=['AC2'], ascending = False)

In [ ]:
df = results_zscore_df.copy()

# Define mapping
old_methods = ['dec', 'kmeans', 'kmeans_pca', 'kmedoids', 'kmedoids_pca', 'ward', 'ward_pca']
new_methods = ['DEC (proposed method)', 'K-Means', 'K-Means_with_pca', 'K-Medoids', 'K-Medoids_with_pca', 'Ward', 'Ward_with_pca']
method_map = dict(zip(old_methods, new_methods))

# Apply mapping
df['method'] = df['method'].map(method_map)


# Ensure columns are correct and values are flattened
df['k'] = df['k'].astype(int)
df['silhouette_score'] = df['silhouette_score'].astype(float)
df['AC1'] = df['AC1'].astype(float)
df['AC2'] = df['AC2'].astype(float)

# Define method groups and colors
methods = ['DEC (proposed method)', 'K-Means', 'K-Means_with_pca', 'K-Medoids', 'K-Medoids_with_pca', 'Ward', 'Ward_with_pca']
base_colors = {
    'K-Means': '#1f77b4',
    'K-Medoids': '#ff7f0e',
    'Ward': '#2ca02c'
}
highlight_color = '#d62728'  # red for 'DEC (proposed method)'

# Color map with pale variants for base methods and shared colors
color_map = {
    'DEC (proposed method)': highlight_color,
    'K-Means': base_colors['K-Means'] + '80',
    'K-Means_with_pca': base_colors['K-Means'],
    'K-Medoids': base_colors['K-Medoids'] + '80',
    'K-Medoids_with_pca': base_colors['K-Medoids'],
    'Ward': base_colors['Ward'] + '80',
    'Ward_with_pca': base_colors['Ward']
}

# Plotting
fig, axs = plt.subplots(3, 1, figsize=(12, 15), sharex=True)
metrics = ['silhouette_score', 'AC1', 'AC2']
ylabels = ['Silhouette Score', 'A(C)1', 'A(C)2']

for ax, metric, ylabel in zip(axs, metrics, ylabels):
    for method in methods:
        subset = df[df['method'] == method]
        x = subset['k'].to_numpy()
        y = subset[metric].to_numpy()
        ax.plot(x, y, label=method,
                color=color_map[method], linewidth=2 if method == 'DEC (proposed method)' else 1.5)
        ax.scatter(x, y, color=color_map[method], s=40)
        if method == 'DEC (proposed method)':
            ax.text(x[-1] + 0.1, y[-1], 'DEC', color=color_map[method],
                    fontsize=12, fontweight='bold', va='center')
    ax.set_ylabel(ylabel, fontsize=12)
    ax.grid(True)

axs[-1].set_xlabel('Number of Clusters (k)', fontsize=12)
fig.suptitle("Evaluation of Clustering on 'Transition-Out-of-Possesion' Data", fontsize=16, fontweight='bold', y=.95)
axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=4, fontsize=10, frameon=False)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()